Setting SmugMug Print Size Keywords with Jupyter and Python
=========================================

![](jupysmug.png)

## Prerequistes

This notebook assumes you have set up your environment to use `smugpyter.py`. 
Refer to this notebook for details on how to do this.

[Getting Ready to use the SmugMug API with Python and Jupyter](https://github.com/bakerjd99/smugpyter/blob/master/notebooks/Getting%20Ready%20to%20use%20the%20SmugMug%20API%20with%20Python%20and%20Jupyter.ipynb)

## Why am I doing this?

Many years ago I wrote a little J verb `smuprintsizes` that computed 
the [largest standard SmugMug print sizes](https://analyzethedatanotthedrivel.org/2010/02/21/assigning-smugmug-print-size-keys/) when given image dimensions 
and the desired [DPI](https://en.wikipedia.org/wiki/Dots_per_inch). I used 
the output of this verb to set aspect ratio keywords for my SmugMug pictures until changes to 
SmugMug, particularly the introduction of [OAuth authentication](https://en.wikipedia.org/wiki/OAuth), broke my little SmugMug API application that called `smugprintsizes`. 

My print size keyword setter broke years ago but many of these keys still show up in my 
["top hundred"](https://conceptcontrol.smugmug.com/keyword) keywords. 

    10x15 4x5 4x6 5x5 5x6.7 5x7 ...
    
Print size keywords were very handy. They made it easy to select paper sizes for one or hundreds of pictures. 
This notebook will use the SmugMug API and Python to compute and set print size keywords.

## The Print Sizes Table

`smugprintsizes` made use of the following table.

     ┌─────┬─────────┬──────────────┐
     │0.7  │17.5 70  │3.5x5 7x10    │
     ├─────┼─────────┼──────────────┤
     │0.8  │20 80    │4x5 8x10      │
     ├─────┼─────────┼──────────────┤
     │0.755│21.2 84.8│4x5.3 8x10.6  │
     ├─────┼─────────┼──────────────┤
     │0.665│24 96    │4x6 8x12      │
     ├─────┼─────────┼──────────────┤
     │0.5  │32 50 128│4x8 5x10 8x16 │
     ├─────┼─────────┼──────────────┤
     │1    │25 64 100│5x5 8x8 10x10 │
     ├─────┼─────────┼──────────────┤
     │0.745│33.5     │5x6.7         │
     ├─────┼─────────┼──────────────┤
     │0.715│35       │5x7           │
     ├─────┼─────────┼──────────────┤
     │0.165│150      │5x30          │
     ├─────┼─────────┼──────────────┤
     │0.4  │160      │8x20          │
     ├─────┼─────────┼──────────────┤
     │0.775│93.5     │8.5x11        │
     ├─────┼─────────┼──────────────┤
     │0.75 │108      │9x12          │
     ├─────┼─────────┼──────────────┤
     │0.77 │130      │10x13         │
     └─────┴─────────┴──────────────┘
     
The first column is the `Short/Long` image aspect ratio rounded to 0.005. The middle column 
lists areas in square inches of the corresponding print sizes in the last column.

This table uses inches but the algorithm doesn't care about units. You can easily
use metric values.

Finding the largest DPI dependent print size is simple matter of:

1. Divide the short image dimension by the long image dimension and round to 0.005.
   This is the aspect ratio.

2. Search for an aspect ratio match in the first column. Many images will not match.
   Quit and return `0z1` for no aspect match. The `0zN` codes are similiar to 
   the `NxM` print sizes codes. This will be important in later notebooks.

3. If a match is found compute the print area required for a given DPI and round to 0.5.

4. Find the index of the largest area in the second column that is greater than or equal to the required 
   area computed in the previous step. If there are not enough pixels no area will meet this criterion.
   Quit and return `0z0` for not enough pixels. 
   
5. If an area is found select and return the corresponding print size in the last column.



An image with dimensions of 2389 x 3344 has enough pixels to make a standard 5x7 inch 360 DPI print. 
It does not  have enough pixels to make a 5x7 inch 720 DPI print. 

Print resolution is a hot button issue for photographers. How many dots (DPI) or pixels (PPI) are 
required depends on many factors, viewing distance, illumination, image colors, paper gloss and so on. 
Human vision tests have demonstrated that young people with excellent eyesight can tell the difference
between 500 DPI and 600 DPI prints. Resolutions beyond 600 DPI are mostly wasted unless you are using loupes or microscopes.
[According to Dr. Optoglass](https://wolfcrow.com/blog/notes-by-dr-optoglass-the-resolution-of-the-human-eye/):

>*If the average reading distance is 1 foot (12 inches = 305 mm), p @0.4 arc minute is 35.5 microns or about 720 ppi/dpi. p @1 arc minute is 89 microns or about 300 dpi/ppi. This is why magazines are printed at 300 dpi – it’s good enough for most people. Fine art printers aim for 720, and that’s the best it need be. Very few people stick their heads closer than 1 foot away from a painting or photograph.*

Digital printers  complicate DPI issues by applying  sophisticated resizing algorithms that can turn low resolution 
originals into plausible higher resolution copies. I've found that 360 DPI is a good starting point for SmugMug prints.
For exceptional images you can simply divide the 360 DPI image dimensions by two for 720 DPI printing. 

## Computing DPI Dependent Print Area

The use of the print size table is clear with the exception of computing the print area required for a given DPI.
`dpi_area` computes DPI dependent print area.

In [1]:
def round_to(n, precision):
    correction = 0.5 if n >= 0 else -0.5
    return int( n/precision+correction ) * precision

def aspect_ratio(height, width, *, precision=0.005):
    return round_to( min(height, width) / max(height, width), precision )

def dpi_area(height, width, *, dpi=360, precision=0.5):
    return round_to( (height * width) / dpi ** 2, precision )

# image pixel dimensions - order is immaterial
height, width = 2389 , 3344

print('aspect ratio %s' % aspect_ratio(height, width))
print('area at 360 dpi %s' % dpi_area(height, width))
print('area at 720 dpi %s' % dpi_area(height, width, dpi=720))

aspect ratio 0.715
area at 360 dpi 61.5
area at 720 dpi 15.5


## Representing the Print Size table

There are many ways to encode the print size table. I am starting with the simplest possible representation: three lists,
one for each column.

The lists must have the same number of items. Eventually, these details will be hidden within a `SmugPyter` subclass 
that manages the details of creating and using print size tables. For now let's build the lists from a simple string.

In [2]:
import smugpyter
smugmug = smugpyter.SmugPyter()

In [3]:
# list of all known small to medium SmugMug print sizes
smug_print_sizes = """
 3.5x5  4x5    4x5.3  4x6    4x8    
 5x5    5x6.7  5x7    5x10   5x30   
 7x10   8x8    8x10   8x10.6 8x12   
 8x16   8x20   8.5x11 9x12   10x10  
 10x13  10x15  10x16  10x20  10x30  
 11x14  11x16  11x28  12x12  12x18  
 12x20  12x24  12x30  16x20  16x24  
 18x24  20x20  20x24  20x30 
"""

# clean up the usual suspects
smug_print_sizes = smugmug.purify_smugmug_text(smug_print_sizes).split()
print(smug_print_sizes)

['3.5x5', '4x5', '4x5.3', '4x6', '4x8', '5x5', '5x6.7', '5x7', '5x10', '5x30', '7x10', '8x8', '8x10', '8x10.6', '8x12', '8x16', '8x20', '8.5x11', '9x12', '10x10', '10x13', '10x15', '10x16', '10x20', '10x30', '11x14', '11x16', '11x28', '12x12', '12x18', '12x20', '12x24', '12x30', '16x20', '16x24', '18x24', '20x20', '20x24', '20x30']


In [4]:
all_aspect_ratios = []
all_print_areas = []

for size in smug_print_sizes:
    height , width = size.split('x')
    height = float(height) 
    width = float(width)
    ratio = aspect_ratio(height, width)
    area = height * width
    all_aspect_ratios.append(ratio)
    all_print_areas.append(area)
    
aspect_ratios = list(set(all_aspect_ratios))
print(aspect_ratios)

[0.7000000000000001, 0.8, 0.755, 0.665, 0.5, 1.0, 0.745, 0.715, 0.165, 0.4, 0.775, 0.75, 0.625, 0.335, 0.6900000000000001, 0.77, 0.395, 0.6, 0.835, 0.785]


In [5]:
# group areas and keys by ratios
gpa = []
gsk = []
for ur in aspect_ratios:
    gp = []
    gk = []
    for ar, pa, sk in zip(all_aspect_ratios, all_print_areas, smug_print_sizes):
        if ur == ar:
            # NIMP: insure these sublist of sorted by ascending area
            gp.append(pa)
            gk.append(sk)
    gpa.append(gp)
    gsk.append(gk)
    
print_areas = gpa
size_keywords = gsk

In [6]:
#aspect_ratios = [0.7, 0.8, 0.755, 0.665, 0.5, 1, 0.745, 0.715, 
#                 0.165, 0.4, 0.775, 0.75, 0.77]
print(aspect_ratios)
print(len(aspect_ratios))

[0.7000000000000001, 0.8, 0.755, 0.665, 0.5, 1.0, 0.745, 0.715, 0.165, 0.4, 0.775, 0.75, 0.625, 0.335, 0.6900000000000001, 0.77, 0.395, 0.6, 0.835, 0.785]
20


In [7]:
#print_areas = [[17.5,70],[20,80],[21.2,84.8],[24,96],[32,50,128],
#               [25,64,100],[33.5],[35],[150],[160],[93.5],[108],[130]]
print(print_areas)
print(len(print_areas))

[[17.5, 70.0], [20.0, 80.0, 320.0], [21.2, 84.8], [24.0, 96.0, 150.0, 216.0, 384.0, 600.0], [32.0, 50.0, 128.0, 200.0, 288.0], [25.0, 64.0, 100.0, 144.0, 400.0], [33.5], [35.0], [150.0], [160.0, 360.0], [93.5], [108.0, 432.0], [160.0], [300.0], [176.0], [130.0], [308.0], [240.0], [480.0], [154.0]]
20


## Minimum Print Size Area

Any image with a `dpi_area` below the minimum print size table area does not have enough pixels to print. 
It's useful to know this value. The following `flatten` function 
from [Recipe 4.14, Python Cookbook 3rd Ed](https://www.safaribooksonline.com/library/view/python-cookbook-3rd/9781449357337/) makes it easy to extract this value.

In [8]:
from collections import Iterable

def flatten(items):
    """Yield items from any nested iterable; see REF."""
    for x in items:
        if isinstance(x, Iterable) and not isinstance(x, (str, bytes)):
            yield from flatten(x)
        else:
            yield x
            
min_print_area = min(list(flatten(print_areas)))
print(min_print_area)

17.5


In [9]:
#size_keywords = [['3.5x5','7x10'],['4x5','8x10'],['4x5.3','8x10.6'],
#                 ['4x6','8x12'],['4x8','5x10', '8x16'],['5x5','8x8','10x10'],['5x6.7'],
#                 ['5x7'],['5x30'],['8x20'],['8.5x11'],['9x12'],['10x13']]
print(size_keywords)
print(len(size_keywords))

[['3.5x5', '7x10'], ['4x5', '8x10', '16x20'], ['4x5.3', '8x10.6'], ['4x6', '8x12', '10x15', '12x18', '16x24', '20x30'], ['4x8', '5x10', '8x16', '10x20', '12x24'], ['5x5', '8x8', '10x10', '12x12', '20x20'], ['5x6.7'], ['5x7'], ['5x30'], ['8x20', '12x30'], ['8.5x11'], ['9x12', '18x24'], ['10x16'], ['10x30'], ['11x16'], ['10x13'], ['11x28'], ['12x20'], ['20x24'], ['11x14']]
20


In [10]:
def print_size_key(height, width, *, no_ratio='0z1', no_pixels='0z0', 
                   min_area=17.5, ppi=360, tolerance=0.000005):
    """
    Compute print size key word from image dimensions. 
    The result is a character string.
    
      key360 = print_size_key(2000, 3000)
      
      # (ppi) is identical to dpi here
      key720 = print_size_key(2000, 3000, ppi=720) 
    """
    
    # basic argument check
    error_message = '(height), (width) must be positive integers'
    if not (isinstance(height, int) and isinstance(width, int)):
        raise ValueError(error_message)
    elif height <= 0 or width <= 0:
        raise ValueError(error_message)
    
    # area must exceed a minimum size
    print_area = dpi_area(height, width, dpi=ppi)
    if print_area < min_area:
        return no_pixels
    
    print_ratio = aspect_ratio(height, width)
    print_key = no_ratio
    for i, ratio in enumerate(aspect_ratios):
        if abs(print_ratio - ratio) <= tolerance:
            print_key = no_pixels
            
            # not enough or more than enough area
            if print_area < print_areas[i][0]:
                break
            elif print_area > print_areas[i][-1]:
                print_key = size_keywords[i][-1]
                break     
            
            for j, area in enumerate(print_areas[i]):
                if area >= print_area and 0 < j:
                    print_key = size_keywords[i][j - 1]
                    break
                    
    return print_key
    
# many sizes available for aspect ratio 1.0
print('3800x3800 at 360 DPI = %s' % print_size_key(3800, 3800))
print('3800x3800 at 720 DPI = %s' % print_size_key(3800, 3800, ppi=720))
print('3000x3000 at 360 DPI = %s' % print_size_key(3000, 3000))
print('2000x2000 at 360 DPI = %s' % print_size_key(2000, 2000))

# not enough pixels
print('500x500 at 360 DPI = %s' % print_size_key(500,500))
print('10x10 at 360 DPI = %s' % print_size_key(10,10)) 

# no ratio 
print('3255x4119 at 360 DPI = %s' % print_size_key(3255, 4119))

3800x3800 at 360 DPI = 10x10
3800x3800 at 720 DPI = 5x5
3000x3000 at 360 DPI = 8x8
2000x2000 at 360 DPI = 5x5
500x500 at 360 DPI = 0z0
10x10 at 360 DPI = 0z0
3255x4119 at 360 DPI = 0z1


## Testing `print_size_key`

The `print_size_key` function seems simple enough but when I see three `break` statements in a loop
I set my bullshit detector to 
[eleven](https://duckduckgo.com/?q=you+tube+loudness+to+eleven&ia=videos&iax=videos&iai=4xgx4k83zzc) and start 
looking for bugs.

In [11]:
# exception throwing blocks rerunning all notebook cells
# print_size_key('not', 'even_wrong') # throw exception

In [12]:
# print_size_key(-2, -3) # throw exception

In [13]:
# print_size_key(0, 50) # throw exception

In [14]:
print('0z0' == print_size_key(1,1))      # not enough pixels
print('0z0' == print_size_key(20,20))    # not enough pixels
print('0z0' == print_size_key(500,500))  # not enough pixels

True
True
True


In [15]:
print('0z1' == print_size_key(2000,2100))  # ratio not in table
print('0z1' == print_size_key(4000,3500))  # ratio not in table
print('0z1' == print_size_key(1000,5000))  # ratio not in table

True
True
True


As `print_size_key` rounds ratios and areas you need slightly more pixels than you might expect 
for a given print size. In practice this is not an issue as digital images usually have
more than enough pixels for small standard size prints.

In [16]:
print('0z0' == print_size_key(int(3.5 * 350), 5 * 350))           # 3.5x5 not enough pixels
print('3.5x5' == print_size_key(int(3.5 * 362), 5 * 362))         # 3.5x5
print('7x10' == print_size_key(7 * 362, 10 * 362))                # 7x10
print('5x6.7' == print_size_key(5 * 362, int(6.7 * 362)))         # 5x6.7
print('8.5x11' == print_size_key(int(8.5 * 362), 11 * 362))       # 8.5x11
print('10x10' == print_size_key(10 * 362, 10 * 362))              # 10x10
print('10x10' == print_size_key(10 * 722, 10 * 722, ppi=720))     # 10x10 at 720 DPI
print('5x30' == print_size_key(5 * 362, 30 * 362))                # 5x30
print('5x10' == print_size_key(5 * 722, 10 * 722, ppi=720))       # 5x10 at 720 DPI

True
True
True
True
True
True
True
True
True


In [17]:
# selected actual SmugMug image dimensions
print(print_size_key(2396,1991))  
print(print_size_key(2585,1736))
print(print_size_key(4573,3259))
print(print_size_key(2800,1999))

0z1
0z1
5x7
5x7


## Calculating Print Size Keys for SmugMug Album Manifest Files

In the first notebook of this series I used the SmugMug API to generate folders and files
containing SmugMug image metadata stored in CSV TAB delimited files. Now I will read these manifest files and
compute print size keys.

In [18]:
import csv

with open('c:\SmugMirror\Places\Overseas\Ghana1970s\manifest-Ghana1970s-Kng6tg-w.txt', 'r') as f:
    reader = csv.DictReader(f, dialect='excel', delimiter='\t')                     
    for row in reader:
        key = row['ImageKey']
        height , width = int(row['OriginalHeight']), int(row['OriginalWidth'])
        size_key = print_size_key(height, width)
        print(key, size_key, height, width)

4wqd5Hr 4x6 3021 2014
K7JKbs8 0z1 2036 3122
nFRxBh2 5x7 2665 3731
xCdD7V8 0z1 2585 1736
sTXnpLm 4x6 2192 3289
VG2s4WG 5x7 3659 2613
kNRs3X8 4x6 1694 2543
Qjs2hr6 4x6 3848 2559
qbXqVgC 4x6 2633 3949
ZdzNXm3 0z1 1162 2506
vF4Bwpg 5x7 2531 3542
7WbqpMj 4x5 3211 2566
2cCVDMK 0z0 1846 2398
36kBgrv 0z1 2396 1991
2FzVqjP 0z0 1887 2398


The print size keys computed by the Python `print_size_key` function match the keys computed by
the J verb `printsizekey`.  

     printsizekey=:3 : 0

     NB.*printsizekey v-- j version of python (print_size_key).
     NB.
     NB. monad:  st =. printsizekey btclManifest
     NB.
     NB.   mf0=. readtd2 'C:\SmugMirror\Places\Overseas\Ghana1970s\manifest-Ghana1970s-Kng6tg.txt'
     NB.   mf1=. readtd2 'C:\SmugMirror\Themes\Diaries\CellPhoningItIn\manifest-CellPhoningItIn-PfCsJz.txt'
     NB.   printsizekey mf0
     NB.   printsizekey mf1
     NB.
     NB. dyad:  st =. iaDpi printsizekey btclManifest
     NB.
     NB.   720 printsizekey mf1

     SMUGPRINTDPI printsizekey y
     :
     NB. image keys and dimensions 
     d=. y {"1~ (0{y) i. ;:'ImageKey OriginalHeight OriginalWidth'
     f=. |: _1&".&> d=. 1 2 {"1 }. d
     'invalid image dimensions' assert 0 < ,f

     NB. default print size keys
     'area ratio'=. (SMUGASPECTROUND,SMUGAREAROUND,x) dpiarearatio f 
     keys=. (#ratio) # s: <NORATIOKEY

     NB. print sizes for image ratios
     pst=.  SMUGASPECTROUND printsizestable SMUGPYTERSIZES
     ast=.  ;0{"1 pst
     m0=.   ratio e. ast
     idx=.  (ast i. ratio) -. #ast
     pst=.  idx { pst

     NB. images without enough pixels
     area=. <"0 m0 # area
     m1=.   (1 {"1 pst) <&.> area
     m2=.   +./&> m1
     keys=. (s: <NOPIXELSKEY) (I. m0 #^:_1 -. m2)} keys

     NB. largest print sizes for enough pixels
     sizes=. ,(I.@lastones&.> m2#m1) {&> 2 {"1 m2#pst
     keys=. sizes(I. m0 #^:_1 m2)} keys

     NB. image keys, print size keys, pixels
     NB. smoutput (<"0  m0 # keys) ,. area ,. pst 
     (s: }.0 {"1 y) , keys , |: s: d 
     )
     
     	 mf0=. readtd2 'c:\SmugMirror\Places\Overseas\Ghana1970s\manifest-Ghana1970s-Kng6tg-w.txt'
         |: printsizekey mf0
     `4wqd5Hr `4x6 `3021 `2014
     `K7JKbs8 `0z1 `2036 `3122
     `nFRxBh2 `5x7 `2665 `3731
     `xCdD7V8 `0z1 `2585 `1736
     `sTXnpLm `4x6 `2192 `3289
     `VG2s4WG `5x7 `3659 `2613
     `kNRs3X8 `4x6 `1694 `2543
     `Qjs2hr6 `4x6 `3848 `2559
     `qbXqVgC `4x6 `2633 `3949
     `ZdzNXm3 `0z1 `1162 `2506
     `vF4Bwpg `5x7 `2531 `3542
     `7WbqpMj `4x5 `3211 `2566
     `2cCVDMK `0z0 `1846 `2398
     `36kBgrv `0z1 `2396 `1991
     `2FzVqjP `0z1 `1887 `2398
	 
The J verb and the Python function use completely different approaches but arrive at
the same result. If you really care about the answer do it more than once and practice relentless verification.

The following functions generalize setting print size keywords for manifest files.

In [19]:
import re

def update_size_keyword(size_keyword, keywords, split_delimiter=';'):
    """
    Update the print size keyword for a single image
    and standardize the format of any remaining keywords.
    Result is a (boolean, string) tuple.
    """
    # basic argument check
    error_message = '(size_keyword), (keywords) must be nonempty strings'
    if not (isinstance(size_keyword, str) and isinstance(keywords, str)):
        raise TypeError(error_message)
    elif len(size_keyword.strip(' ')) == 0:
        raise ValueError(error_message)
        
    if len(keywords.strip(' ')) == 0:
        return (False, size_keyword)
    
    inkeys = [s.strip().lower() for s in keywords.split(split_delimiter)]
    if 0 == len(inkeys):
        return (False, size_keyword)
    
    outkeys = [size_keyword]
    for inword in inkeys:
        # remove any existing print size keys
        if re.match(r"\d+(\.\d+)?[xz]\d+(\.\d+)?", inword) is not None:
            continue
        else:
            outkeys.append(inword)
            
    # return unique sorted keys
    outkeys = sorted(list(set(outkeys)))
    return (set(outkeys) == set(inkeys), (split_delimiter+' ').join(outkeys))

def print_keywords(manifest_file):
    """
    Set print size keywords for images in album manifest file.
    Result is a list of dictionaries in (csv.DictWriter) format.
    """
    changed_keywords = []
    image_count = 0
    with open(manifest_file, 'r') as f:
        reader = csv.DictReader(f, dialect='excel', delimiter='\t')                     
        for row in reader:
            image_count += 1
            key = row['ImageKey']
            height , width = int(row['OriginalHeight']), int(row['OriginalWidth'])
            size_key = print_size_key(height, width)
            same, keywords = update_size_keyword(size_key, row['Keywords'])
            if not same:
                changed_keywords.append({'ImageKey': key, 'AlbumKey': row['AlbumKey'],
                                       'FileName': row['FileName'], 'Keywords': keywords,
                                       'OldKeywords': row['Keywords']})
    return (image_count, changed_keywords)

In [20]:
print_keywords('c:\SmugMirror\Places\Overseas\Ghana1970s\manifest-Ghana1970s-Kng6tg-w.txt')

(15,
 [{'AlbumKey': 'Kng6tg',
   'FileName': 'boy corn harvest tamale ghana 1976.jpg',
   'ImageKey': '4wqd5Hr',
   'Keywords': '4x6; boy; corn; ghana; harvest; tamale',
   'OldKeywords': 'boy; corn; harvest; tamale; ghana'},
  {'AlbumKey': 'Kng6tg',
   'FileName': 'corn harvest students tamale ghana.jpg',
   'ImageKey': 'K7JKbs8',
   'Keywords': '0z1; corn; ghana; harvest; students; tamale',
   'OldKeywords': 'corn; harvest; students; tamale; ghana'},
  {'AlbumKey': 'Kng6tg',
   'FileName': 'depression catholic school ghana 1975.jpg',
   'ImageKey': 'nFRxBh2',
   'Keywords': '5x7; catholic; depression; ghana; school',
   'OldKeywords': 'depression; catholic; school; ghana'},
  {'AlbumKey': 'Kng6tg',
   'FileName': 'dry season sun [9622633].jpg',
   'ImageKey': 'xCdD7V8',
   'Keywords': '0z1; dimmed; dry; season; sun',
   'OldKeywords': 'dimmed; dry; season; sun'},
  {'AlbumKey': 'Kng6tg',
   'FileName': 'ghana cusos beach 1975.jpg',
   'ImageKey': 'sTXnpLm',
   'Keywords': '4x6; beach

## Testing `update_size_keyword`

In [21]:
# update_size_keyword('4x5', 3)  # throw exception

In [22]:
# update_size_keyword('', ' ok; but; size; key; bad')  # throw exception

In [23]:
print('4x6' == update_size_keyword('4x6', '     ')[1])
print('4x6; boo' == update_size_keyword('4x6', 'boo')[1]) 
print('4x6; aha; boo; boys' == update_size_keyword('4x6', 'aha; boo; BOO; boo; boys')[1])
print('4x6' == update_size_keyword('4x6', '5x7; 8x12; 3x4; 3.5x5')[1]) 
print('4x6; boo; home; yo' == update_size_keyword('4x6', '5x7; 8x12; 3x4; 3.5x5; yo; yo; home; BOO')[1])
print(update_size_keyword('4x6', '4x6; boo; hoo; too')[0]) # no keyword changes

True
True
True
True
True
True


## Posting SmugMug Print Size Keywords

The next step is to post the computed print size keywords to SmugMug. For this, we need
an API call that sets keywords. The `SmugPyter` class does have a keyword setting function. 
We will have to fake it.

In case you are wondering, faking it is a fundamental skill that all programmers must master.
Remember how Scotty in the original Star Trek series constantly told Kirk that he couldn't
sustain high warp without wreaking the Enterprise but somehow always managed to do it and walk away
intact. Sure the Enterprise wasn't designed for the stresses it was forced to endure but Scotty
hacked it on the fly. 

A lot of programming is like that. You're working with half-baked buggy tools that will not
sustain warp but you have to pull it off. Be grateful you're not dodging photon torpedoes.

In [24]:
import os 

def album_id_from_file(filename):
    """
    Extracts the (album_id, name, mask) from file names. 
    Depends on file naming conventions.
    
        album_id_from_file('c:\SmugMirror\Places\Overseas\Ghana1970s\manifest-Ghana1970s-Kng6tg-w.txt')    
    """
    mask, album_id, name = filename.split('-')[::-1][:3]
    mask = mask.split('.')[0]
    return (smugmug.case_mask_decode(album_id, mask), name, mask)

manifest_file = 'c:\SmugMirror\Places\Overseas\Ghana1970s\manifest-Ghana1970s-Kng6tg-w.txt'
album_id_from_file(manifest_file)

('Kng6tg', 'Ghana1970s', 'w')

In [25]:
def write_keyword_changes(manifest_file):
        """
        Write TAB delimited file of changed metadata.
        Return album and keyword (image_count, change_count) tuple.
        """
        image_count, keyword_changes = print_keywords(manifest_file)
        change_count = len(keyword_changes)
        if change_count == 0:
            return (image_count, 0)
            
        album_id, name, mask = album_id_from_file(manifest_file)
        path = os.path.dirname(manifest_file)
        
        changes_name = "changes-%s-%s-%s" % (name, album_id, mask)
        changes_file = path + "/" + changes_name + '.txt'
        
        keys = keyword_changes[0].keys()
        with open(changes_file, 'w', newline='') as output_file:
            dict_writer = csv.DictWriter(output_file, keys, dialect='excel-tab')
            dict_writer.writeheader()
            dict_writer.writerows(keyword_changes)
            
        return(image_count, change_count)
            
write_keyword_changes(manifest_file)

(15, 14)

In [73]:
def update_all_keyword_changes_files(root):
    """
    Scan all manifest files in local directories and
    generate TAB delimited CSV keyword changes files.
    """
    total_images , total_changes = 0 , 0
    pattern = "manifest-"
    alist_filter = ['txt'] 
    for r,d,f in os.walk(root):
        for file in f:
            if file[-3:] in alist_filter and pattern in file:
                image_count, change_count = write_keyword_changes(os.path.join(root,r,file))
                total_images += image_count
                total_changes += change_count
    print('image count %s, change count %s' % (total_images, total_changes))

In [74]:
# %timeit update_all_keyword_changes_files('c:\SmugMirror')
update_all_keyword_changes_files('c:\SmugMirror')

image count 4241, change count 2716


## Issuing SmugMug API `PATCH` Requests

Now that the `CSV` change files are ready the next step is to read them and reset keywords. You can do this with a SmugMug `PATCH` request. 

My attempts to issue `PATCH` requests did not meet with a lot of success until I traded a few emails with the SmugMug API support team at `api@smugmug.com`. They advised me to turn off redirects. It was a simple parameter setting but it would have taken me days to figure it on my own. Kudos to the excellent API support at SmugMug.

In [33]:
import requests
import json
from requests_oauthlib import OAuth1

In [43]:
auth = OAuth1(smugmug.consumer_key, smugmug.consumer_secret, 
              smugmug.access_token, smugmug.access_token_secret, smugmug.username)

In [42]:
# attempt to set keywords
r = requests.patch(url='https://api.smugmug.com/api/v2/image/8rjZsTB',
                   auth=auth,
                   data=json.dumps({"Keywords": "these; are; brand; spanking; new; keywords"}),
                   headers={'Accept':'application/json','Content-Type':'application/json'},
                   allow_redirects=False)



In [55]:
def change_image_keywords(image_id, keywords):
    r = requests.patch(url='https://api.smugmug.com/api/v2/image/' + image_id,
                       auth=auth,
                       data=json.dumps({"Keywords": keywords}),
                       headers={'Accept':'application/json','Content-Type':'application/json'},
                       allow_redirects=False)
    if r.status_code != 301:
        raise Exception("Not what the doctor ordered")
    
    return 'changed'
        

In [56]:
change_image_keywords('8rjZsTB', 'more; new; keywords; ehh')

'changed'

In [63]:
def change_keywords(changes_file):
    """
    Set print size keywords for images in album changes file.
    """
    image_count = 0
    with open(changes_file, 'r') as f:
        reader = csv.DictReader(f, dialect='excel', delimiter='\t')                     
        for row in reader:
            image_count += 1
            key = row['ImageKey']
            keywords = row['Keywords']
            print(key, keywords)
            change_image_keywords(key, keywords)
    return image_count

change_keywords('c:/SmugMirror/Other/utilimages/changes-utilimages-GMLn9k-1k.txt')

mVp5QST 0z0; forecast
QMZfvjP 0z0; book; bullshit; china
pmw9LBj 0z0; emailteaser
bMCTPM3 0z0; ending
Gq3WvQ9 0z0; composite; sdo; transit; venus
4ZxhvXn 0z0; morley; px; triangle
8rjZsTB 0z0; already; damn; request; show; the
trnDW26 0z0; pxouter; space; treaty
L5RD8CV 0z0; toronto
VTHCgxD 0z0; headupasslogo
Q6CM5Rx 0z0; box; idiot
RgccVcz 0z0; martingardnerpuzzle
QRdsDd2 0z0; buyers; grinding; nar
T8tX2bX 0z0; diagram; dork; fall; geek; nerd; reliable; venn
VhbkTxq 0z0; grin; obama
VW88d2q 0z0; olivia; wilde
dZsBCHG 0z0; proofgeneral
vWJ24xK 0z0; socializedmedicine
9nmTkJJ 0z0; artist; created; hong; jobs; memorials; steve; stevejobsmemorial
pJkxfxb 0z0; politicians; uranus
Rt3PfWj 0z1; analystdiagram; diagram
2FVN936 0z0; anathem
4VxzJWb 0z0; aplbadboys
qNTb7XL 0z0; apltypeball
xws2kD3 0z0; awesomeevil
Lq3N689 0z0; believer
4DVCtH2 0z0; black; swan
cNF63Lh 0z0; diamond; icon
TnGvMkw 0z0; art; blue; clip; yang; yin
PpQz2jC 0z0; bonebridge
ZNStb7v 0z0; bpoilshores
LKVRXF5 0z0; bullshi

107

Once an album's print size keywords have been changed regenerating the print size keywords changes files should result in a file with no pending changes.

*Note: posted keyword changes appear to become immediately active on SmugMug but immediately re-pulling them returns the prior keyword list. This may be an update issue. I will check later.*

In [64]:
write_keyword_changes('c:/SmugMirror/Other/utilimages/manifest-utilimages-GMLn9k-1k.txt')

(107, 107)

In [71]:
def update_all_keyword_changes(root):
    """
    Scan all changes files in local directories
    and apply keyword changes.
    """
    total_changes = 0
    pattern = "changes-"
    alist_filter = ['txt'] 
    for r,d,f in os.walk(root):
        for file in f:
            if file[-3:] in alist_filter and pattern in file:
                change_count = change_keywords(os.path.join(root,r,file))
                total_changes += change_count
    print('change count %s' % total_changes)

In [75]:
# takes awhile to plow through thousands of updates
# update_all_keyword_changes('c:\SmugMirror')

## Setting a `geotagged` Keyword

Now that we can easily set keywords. It's a simple matter to scan the manifest files and set a `geotagged` keyword for all images that have nonzero latitude and longitude. The most common latitude, longitude and altitude value in the manifest files is the default `(0,0,0)`. 
If you [look at a map](https://www.google.com/maps/place/0%C2%B000'00.0%22N+0%C2%B000'00.0%22E/@-2.4635807,4.1955676,4.5z/data=!4m2!3m1!1s0x0:0x0?hl=en) 
you'll see this coordinate is in Atlantic ocean off the west coast of Africa. I have taken exactly zero pictures at this location.

In [81]:
def geotag_images(manifest_file, split_delimiter=';'):
    """
    Sets a geotagged keyword for images with nonzero latitude and longitude.
    """
    change_count = 0
    with open(manifest_file, 'r') as f:
        reader = csv.DictReader(f, dialect='excel', delimiter='\t')                     
        for row in reader:
            key = row['ImageKey']
            latitude = float(row['Latitude'])
            longitude = float(row['Longitude'])
            if latitude != 0.0 or longitude != 0.0:
                keywords = row['Keywords']
                inkeys = [s.strip().lower() for s in keywords.split(split_delimiter)]
                inkeys.append('geotagged')
                outkeys = sorted(list(set(inkeys)))
                same, new_keywords = (set(outkeys) == set(inkeys), (split_delimiter+' ').join(outkeys))
                if not same:
                    change_count += 1   
                    print(key, new_keywords)
                    change_image_keywords(key, new_keywords)
    return change_count

geotag_images('c:\SmugMirror\Places\Overseas\Ghana1970s\manifest-Ghana1970s-Kng6tg-w.txt')

0

In [80]:
def set_all_geotags(root):
    """
    Scan all manifest files in local directories and set
    geotags for images with nonzero latitude or longitude.
    """
    total_changes = 0
    pattern = "manifest-"
    alist_filter = ['txt'] 
    for r,d,f in os.walk(root):
        for file in f:
            if file[-3:] in alist_filter and pattern in file:
                change_count = geotag_images(os.path.join(root,r,file))
                total_changes += change_count
    print('change count %s' % total_changes)
    
set_all_geotags('c:\SmugMirror')

7ZdVjvN 0z0; geotagged
2BCNG5j 0z0; geotagged
wBbtH9k 0z0; geotagged
pbMWdKB 0z0; geotagged
TLpr62M 0z0; geotagged
HkmQrfT 0z0; geotagged
PnfVpRF 0z0; geotagged
T9xqf7d 0z0; geotagged
BLN79Hq 0z0; geotagged
Rnpz5T8 0z0; geotagged
JJxkwvz 0z0; geotagged
KNFHRt3 0z0; geotagged
Vdm2Qts 0z0; geotagged
j38Cs8s 0z0; geotagged; photo20nov20012c20122017203920pm
nxR8BLb 0z0; geotagged; photo20nov20012c20122018201520pm
7KBL4r9 4x6; casper; geotagged; house; wyoming
L3wdp3r 0z1; aileen; dad; geotagged; house; livingston; mom; montana; steve
KpZfbcG 5x7; after; casper; geotagged; gert; hazel; porch
WTDx5SD 0z0; geotagged; house; june; move; paradise; valley
jL9PP4B 5x7; alberta; awhile; geotagged; grandparent; lived; ora; rare
RRp97fB 0z0; geotagged; house; lake; salt
rPtPRLT 4x5; cart; frank; friend; geotagged; golf; livingston
KpZfbcG 5x7; after; casper; geotagged; gert; hazel; porch
4sXbrtb 0z0; baker; cart; geotagged; golf; grandpa; livingston; montana
qCpDbBp 4x6; dog; geotagged; helen; lady;

psZZKJK 4x6; dot; fresh; geotagged; jerky; road; tending
nvJQPqd 0z1; after; general; geotagged; karl; marx; park; san; sherman
qBfDkV4 0z0; center; geotagged; getty; museum
ZNVbXpr 0z0; geotagged; glacier; live; million; point; reputation; valley; yosemite
gzMGBGv 4x6; canyon; geotagged; golden; stretch; trail; valley; weekend
gppp4sQ 4x6; bridge; crissy; field; gate; geotagged; golden
3XbMZ3D 5x7; diego; flores; geotagged; jacob; point; san
xpKVWR4 4x6; chiriaco; geotagged; jacob; me; patton; statue; summit
tB898PX 0z0; geotagged; joshua; park; sun; sunset; throne; tree; watching
GMj38Ch 0z0; american; geotagged; jpl; la; league; voyager; weekend
HhnLrrb 4x6; canyon; fall; geotagged; king; park; river
SZv6kc8 5x7; geotagged; governator; hollywood; kneeling; looking; print; walking
p8kRqWw 0z1; bird; geotagged; grove; johnson; lady; panorama; park; tree
t9FWG27 5x7; borrego; geotagged; hotel; inn; mali; valley
k4xvcVT 5x7; chair; devil; geotagged; mali; punch; trail
vmxvpFR 0z0; geota

2jCXDxZ 0z0; appreciation; geotagged; warthog
CjkVmvX 0z0; expo; fair; geotagged; idaho; welcome
68prrNP 0z0; dog; fair; geotagged; sculpture; wicker
Tx7bhbk 8x12; balloon; eclipse; geotagged; spanish; yellow
qsncfsx 0z1; alexandra; bridge; geotagged; org; ottawa; panorama; wiki; wikipedia
wJtJZh3 4x6; arboretum; foreground; geotagged; hill; ottawa; southeast; tobaggon
Gffgr6J 0z0; atop; canadian; constructed; designed; diefenbunker; geotagged; museum; ottawa
m4kQB5V 0z1; ball; geotagged; ottawa; sculpture; throw; winter
4XJCn6k 4x6; bench; falling; geotagged; lamp; scene; simplify; snowstorm
VdSnxmZ 4x8; canonical; geotagged; justice; supreme
ZjcZtqj 0z0; bridge; club; geotagged; hunt; span
7Lnzb45 5x6.7; apartment; building; geotagged; iphone; jacob; sharing; student
6w6trqJ 5x7; geotagged; me; meadowlands; nepean
xSH7SrV 0z0; geotagged; hill; me; mirror; parking; parliament
VQGZ5nX 4x6; ball; council; fall; geotagged; near; nrc
f3jSdzk 4x6; geotagged; latitude; longitude; ottawa; ri

4sXbrtb 0z0; baker; cart; geotagged; golf; grandpa; livingston; montana
qKvfkzb 0z0; coalition; gay; geotagged; green; loggers
ZwSxzcm 4x6; devils; evelyn; geotagged; helen; jacob; slide
n836MFp 0z0; geotagged; glacier; going; helen; overlooking; park; road; sun
SLqbkCb 0z1; attitude; creek; geotagged; helen; montana; pine; trail
mf9T8P7 5x7; fork; geotagged; helen; madison; montana; rattlesnake; sign
QxgGNrX 0z1; buses; capital; geotagged; helena; rally
zK4dFHz 0z0; geotagged; glacier; horse; jacob
gjq3ntP 0z1; boot; city; evelyn; frank; geotagged; helen; hill; jacob; virginia
KvnrZ3q 0z0; enjoying; geotagged; glacier; helen; jacob; lake; park
HVH3nt6 0z1; geotagged; lake; lines; missoula; shore
56gDcbm 8x12; clark; exit; geotagged; lewis; trail
qLJDnrn 0z0; geotagged; livingston; locomotive; montana; train; yard
G62QRHB 8x12; down; geotagged; lone; mountain; peak; tram; view
Pdcz2w2 0z1; geotagged; lone; panorama; peak; summit
CMgVvdG 0z0; geotagged; lone; peak; tram; truck
t4xtcPH 5

6Mg49bv 3.5x5; geotagged; lincoln; springfield; standing; tomb
N6hRBfj 0z0; geotagged; lone; peak; selfie
CsNWsKZ 4x6; france; geotagged; louvre; paris
qHKPJrb 4x6; geotagged; lusaka; me; square
brLPQQK 0z1; geotagged; machu; me; overlooking; peru; picchu; spectacular; visited
gHDFVRV 4x6; geotagged; great; mali; me; mountain; sign; smokey
Nmc5SNX 0z0; evans; geotagged; mali; mount; sign
BmH3rsM 0z0; geotagged; mali; nevada; peak; summit; wheeler
282bpgs 4x6; geotagged; grand; hotel; lion; mgm; road; standing
8BL58DW 4x6; geotagged; iran; me; mom; shustar; steve; waterworks
vpxMJ2C 5x7; evans; geotagged; mount; summit
FpWznRp 0z0; geotagged; mount; rushmore
LH4ZDnf 4x6; geotagged; hidden; joshua; me; near; park; valley
KGzxSRq 4x6; bridge; cincinnati; geotagged; me; people; purple; river
5Hbrq5k 0z0; geotagged; oslo; sculpture; vigeland
MKHtcD5 5x7; dome; geotagged; me; mountain; observatory; palomar; telescope
SPmfzKr 5x7; dinosaur; fossils; geotagged; photobombing
PcQ9szG 4x6; except

4z2zxXT 5x6.7; called; foods; geotagged; iphone; mahin; morning; whole
H7sC9Dw 5x6.7; geotagged; iphone; mahin; mall; santa
hjDMGPh 0z0; berkeley; butte; geotagged; mali; overlook; pit
kg96d5v 0z0; beside; boise; eagle; geotagged; mali; river
9r3rffD 5x6.7; elephant; geotagged; iphone; mali; pool; rock; year
PqLMkVJ 0z0; cafe; city; foodie; geotagged; mali; south
MqJk7RC 5x6.7; fountain; geotagged; iphone; mali; plaza; saint; wedding
t6HJdVt 0z0; dumpty; geotagged; humpty; mali; shadow
5hLLMQM 0z0; geotagged; hacinda; lunch; mali; restaurant
HBPTcwJ 5x6.7; forest; geotagged; mahin; mali; park; rink; skating
Tv3sspT 5x6.7; barbeque; geotagged; iphone; makeshift; mali; memorial; weekend
WW8K3H5 5x7; geotagged; mali; mimic; sculpture; street
5KTSrsr 0z0; bistro; geotagged; history; mali; missouri; museum
nHWvxqk 0z0; cahokia; geotagged; iphone; mali; monk; mound; saint; visible
2WxvCh2 0z0; geotagged; maintenance; mali; road; tesuque; trail
bgpqCSX 5x5; frame; geotagged; mali; orange; scu

RqDWQf6 4x6; clingmans; dome; geotagged; mali; november; visited
PgmZSqs 4x6; day; downtown; geotagged; mali; park; roanoke; square
8PxRFZV 0z0; died; garfield; geotagged; historic; house; me; month; site
gHDFVRV 4x6; geotagged; great; mali; me; mountain; sign; smokey
KGzxSRq 4x6; bridge; cincinnati; geotagged; me; people; purple; river
rDTfWwD 4x5; childhood; cincinnati; geotagged; home; standing; supreme; taft
MfBfX2K 4x6; dune; geotagged; landscape; me; newmexico; sand; white
NDGnZK3 4x6; circle; downtown; geotagged; indiana; indianapolis; monument; museum
JvZb6gp 5x7; bluffton; geotagged; nest; tent
TJ4zNNF 4x6; airforce; dayton; display; exhibit; geotagged; museum; national
sxB7ctZ 4x6; geotagged; mexico; mountain; platform; site; sky; taking
ScLCmBS 4x6; farm; feeling; geotagged; horse; indiana; road; sun
wL3JG92 4x6; field; fort; geotagged; park; parkview; visited; wayne
6CWrPkv 4x6; admired; always; closing; geotagged; get; liquor; save
q2nJG9N 4x6; animal; building; geotagged;

rjw43Vj 5x7; bermuda; corner; george; geotagged; saint
8T7r8Br 0z1; bermuda; blue; geotagged
Xg8F74t 0z1; church; geotagged; unfinished
c2xX7nK 0z1; crater; deep; diving; geotagged; nuclear; opening; ruth
DZBv62w 4x6; air; atoll; enewetak; geotagged
6RS9q6W 0z0; enewetak; geotagged; lab; looking; marine; mid; south; tower
TRNFWpg 0z1; atoll; enewetak; geotagged; looking; next; northern; south; standing
WCj3TFn 0z0; approaching; dome; geotagged; runit
F5qPp7d 4x6; air; enewetak; geotagged; lab; marine; mid; pacific
RmfpqxR 4x6; christ; geotagged; janerio; mountain; redeemer; rio; statue
CPV5RwW 5x7; brasilia; brazil; congress; geotagged; me; ruth; tower
gBBQC9B 4x6; approximate; cuzco; geotagged; great; matched; online; wall
brLPQQK 0z1; geotagged; machu; me; overlooking; peru; picchu; spectacular; visited
6fzX7cV 4x6; geotagged; locks; miraflores; panama; ruth
3NwPvnH 4x6; air; airport; arriving; geotagged; luangwa; malawi; park
BWV3s5C 4x6; balloon; curio; geotagged; looking; market
H

gp3ZQw6 0z0; geotagged; pueblo; tableau; taos
Sns97pr 0z0; geotagged; payments; tax
xPVr8Hn 0z1; geotagged; mammoth; tourists; vintage; yellowstone
94QVfm8 0z0; geotagged; plateau; west; yellowstone
FRGbqKv 0z0; antelope; creek; geotagged; parking
LNb3nW2 0z1; center; city; creek; geotagged; panorama
Dq2KDP4 0z1; bed; breakfast; devils; geotagged; tower
hkGXDKT 0z0; fishing; fort; geotagged; lodge; smith
Kqncq2G 8x12; campground; geotagged; green; overlook; river
m86LFWk 0z0; cabin; geotagged; josie; utah
2RzzfGv 0z0; dell; geotagged; lunch
dqRP6tN 0z0; dinosaur; geotagged; mali; monument; sign
hq2RcBq 4x5; fish; fountain; geotagged; mali; pocatello
JV9wRXj 4x6; geotagged; great; lake; mali; salt; tasting
TJF2nD3 0z1; building; geotagged; mammoth; site
bS85PPb 0z1; crazy; geotagged; horse; memorial
jbc3cBX 5x7; devils; geotagged; tower
Nmc5SNX 0z0; evans; geotagged; mali; mount; sign
rBNB49N 4x5; dinosaur; geotagged; mali; pink
vpxMJ2C 5x7; evans; geotagged; mount; summit
FpWznRp 0z0; 

## Next on the Agenda!

Now that I have worked through a proof of concept the next notebook will condense and refine the code in this notebook into a `SmugPyter` print size keyword setting subclass.

Remember, always [Analyze the Data not the Drivel](https://analyzethedatanotthedrivel.org/).

John Baker, Meridian Idaho